#Initial Exploration
####* [Table of Contents](#Table-of-Contents:)

In [79]:
import pandas as pd
import numpy as np
import math
import re
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
!ls atusdata/

activity_codes.pdf  atusact_2013.zip    atuscps_2013.zip    atusresp_2013.zip   atusrost_2013.zip   atusrostec_2013.zip atussum_2013.zip    atususersguide.pdf  atuswho_2013.zip    data_dictionary.pdf


##Relevant links:
- [activity_codes.pdf](atusdata/activity_codes.pdf)

- [atususersguide.pdf](atusdata/atususersguide.pdf)

- [data_dictionary.pdf](atusdata/data_dictionary.pdf)

In [5]:
def open_atus_zip(datafile):
    #datafile = 'atussum_2013'
    summary = None
    with zipfile.ZipFile('atusdata/{}.zip'.format(datafile), 'r') as z:
        f = z.open('{}.dat'.format(datafile))
        return pd.read_csv(f)
summary = open_atus_zip('atussum_2013')
summary.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
variables = sorted(list(summary.columns.values)[:24])
meanings =  ['metropolitan_status',
             'education_level',
             'is_hispanic',
             'race',
             'age_edited',
             'weekly_hours_worked',
             'employment_status',
             'has_multiple_jobs',
             'is_student',
             'school_level',
             'gender',
             'partner_employed',
             'household_children',
             'work_status',
             'weekly_earnings_main',
             'is_holiday',
             'partner_work_status',
             'partner_present',
             'eldercare_minutes',
             'childcare_minutes',
             'youngest_child_age',
             'date',
             'atus_final_weight',
             'atus_case_id']

summary_columns = {key: meanings[i] for i, key in enumerate(variables)}
meanings_unsorted = [summary_columns[key] for key in summary.columns.values[:24]]
# meanings_unsorted.extend(summary.columns.values[24:])

In [7]:
# summary = pd.read_csv("atusdata/atussum_2013.dat")
summary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 36.0 MB


In [8]:
summary.columns

Index(['tucaseid', 'TUFINLWGT', 'TRYHHCHILD', 'TEAGE', 'TESEX', 'PEEDUCA',
       'PTDTRACE', 'PEHSPNON', 'GTMETSTA', 'TELFS', 
       ...
       't181501', 't181599', 't181601', 't181801', 't189999', 't500101',
       't500103', 't500105', 't500106', 't500107'],
      dtype='object', length=413)

Pertinent columns:

* TUFINLWGT - statistical weight of respondent
* TRYHHCHILD - age of youngest child in household
* TEAGE - age of respondent
* TESEX - sex of respondent
* TELFS - working status of respondent
* TRCHILDNUM - number of children in household

In [9]:
summary.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [10]:
import parse_activities as pa
#import importlib
#importlib.reload(pa)
#del pa
pa.atus_codes_list
pa.codes_dict
z = summary_columns.copy()
z.update(pa.codes_dict)
summary_columns = z
summary_columns
code_meanings = [ pa.codes_dict[i] for i in summary.columns.values[24:]]
meanings_unsorted.extend(code_meanings)
meanings_unsorted
# summary4 = summary.rename(columns=pa.codes_dict)
# summary4.head()
# summary5 = summary4
# summary5.columns = pd.MultiIndex.from_tuples(list(zip(meanings_unsorted, summary.columns)))
# summary5.head() # .columns.values

['atus_case_id',
 'atus_final_weight',
 'youngest_child_age',
 'age_edited',
 'gender',
 'education_level',
 'race',
 'is_hispanic',
 'metropolitan_status',
 'employment_status',
 'has_multiple_jobs',
 'work_status',
 'is_student',
 'school_level',
 'partner_present',
 'partner_employed',
 'weekly_earnings_main',
 'household_children',
 'partner_work_status',
 'weekly_hours_worked',
 'date',
 'is_holiday',
 'eldercare_minutes',
 'childcare_minutes',
 'Sleeping',
 'Sleeplessness',
 'Washing, dressing and grooming oneself',
 'Grooming, n.e.c.*',
 'Health-related self care',
 'Self care, n.e.c.*',
 'Personal/Private activities',
 'Interior cleaning',
 'Laundry',
 'Sewing, repairing, & maintaining textiles',
 'Storing interior hh items, inc. food',
 'Housework, n.e.c.*',
 'Food and drink preparation',
 'Food presentation',
 'Kitchen and food clean-up',
 'Food & drink prep, presentation, & clean-up, n.e.c.*',
 'Interior arrangement, decoration, & repairs',
 'Building and repairing furniture

In [11]:
summary2 = summary.rename(columns=summary_columns)
summary3 = summary2
summary3.columns = pd.MultiIndex.from_tuples(list(zip(summary.columns, meanings_unsorted)))
summary3.head() # .columns.values

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
,atus_case_id,atus_final_weight,youngest_child_age,age_edited,gender,education_level,race,is_hispanic,metropolitan_status,employment_status,...,Travel related to volunteering,"Travel related to volunteer activities, n.e.c.*",Travel related to phone calls,Security procedures related to traveling,"Traveling, n.e.c.*",Insufficient detail in verbatim,Missing travel or destination,"Respondent refused to provide information/""none of your business""",Gap/can't remember,Unable to code activity at 1st tier
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# summary2 = summary.rename(columns=summary_columns)
# summary3 = summary2
# summary3.columns = pd.MultiIndex.from_tuples(list(zip(meanings_unsorted, summary.columns)))
# summary3.head() # .columns.values

In [13]:
adults_crit = summary.TEAGE >= 18
no_children_crit = summary.TRCHILDNUM == 0

In [14]:
adults = summary[adults_crit]
adults.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10953 entries, 0 to 11384
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 34.6 MB


In [15]:
people_with_no_children = summary[no_children_crit]
people_with_no_children.info()
len(people_with_no_children)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6481 entries, 2 to 11381
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 20.5 MB


6481

In [16]:
adults_with_no_children = summary[adults_crit & no_children_crit]
adults_with_no_children.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6481 entries, 2 to 11381
Columns: 413 entries, tucaseid to t500107
dtypes: float64(1), int64(412)
memory usage: 20.5 MB


In [17]:
def weighted(df, col):
    """
    Pass in dataframe and a name of column (str) to received statistically weighted number based on TUFINLWGT
    Returns a dataframe with a weighted values column as well as a weighted average
    """
    data = df[['TUFINLWGT', col]]
    data[str(col+'_wgt')] = data.TUFINLWGT * data[col]
    avg = data[str(col+'_wgt')].sum() / data.TUFINLWGT.sum()
    return data, avg

def weighted_col(df, col):
    """
    Pass in dataframe and a name of column (str) to received statistically weighted column based on TUFINLWGT
    """
    data = df[['TUFINLWGT', col]]
    data[str(col+'_wgt')] = data.TUFINLWGT * data[col]
    return data[str(col+'_wgt')]

data = adults_with_no_children[['TUFINLWGT', 't120303']]
data.head()
data2, avg = weighted(data, 't120303')
data = data.rename(columns={"TUFINLWGT": "weight", "t120303": "minutes"})

data2.head()
avg

190.25402840855642

In [18]:
data['weighted_minutes'] = data.weight * data.minutes

In [19]:
data.head()

,weight,minutes,weighted_minutes
2,10377056.507734,60,6.226234e+08
3,7731257.992805,65,5.025318e+08
4,4725269.227067,90,4.252742e+08
5,2372791.046351,270,6.406536e+08
6,5671341.270490,244,1.383807e+09


In [20]:
# Minutes on average spent watching TV (unweighted) - DO NOT USE
data.minutes.sum() / len(data)

211.67427866070051

In [21]:
# Minutes on average spent watching TV (weighted)
data.weighted_minutes.sum() / data.weight.sum()

190.25402840855642

In [22]:
def average_minutes(data, activity_code):
    activity_col = "t{}".format(activity_code)
    data = data[['TUFINLWGT', activity_col]]
    data = data.rename(columns={"TUFINLWGT": "weight", activity_col: "minutes"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [23]:
sleeping = average_minutes(adults_with_no_children, "010101")
sleepless = average_minutes(adults_with_no_children, "010102")
sleeping, sleepless, (sleeping + sleepless) / 60 # hours

(521.0183492943637, 4.0328730764962595, 8.7508537061809992)

In [24]:
def activity_columns(data, activity_code):
    """For the activity code given, return all columns that fall under that activity."""
    col_prefix = "t{}".format(activity_code)
    return [column for column in data.columns if re.match(col_prefix, column)]

In [25]:
activity_columns(summary, "0101")

['t010101', 't010102']

In [26]:
def average_minutes2(data, activity_code):
    cols = activity_columns(data, activity_code)
    activity_data = data[cols]
    activity_sums = activity_data.sum(axis=1)
    data = data[['TUFINLWGT']]
    data['minutes'] = activity_sums
    data = data.rename(columns={"TUFINLWGT": "weight"})
    data['weighted_minutes'] = data.weight * data.minutes
    return data.weighted_minutes.sum() / data.weight.sum()

In [27]:
pa.categories

{'t01': 'Personal Care',
 't02': 'Household Activities',
 't03': 'Caring For & Helping Household (HH) Members',
 't04': 'Caring For & Helping Nonhousehold (NonHH) Members',
 't05': 'Work & Work-Related Activities',
 't06': 'Education',
 't07': 'Consumer Purchases',
 't08': 'Professional & Personal Care Services',
 't09': 'Household Services',
 't10': 'Government Services & Civic Obligations',
 't11': 'Eating and Drinking',
 't12': 'Socializing, Relaxing, and Leisure',
 't13': 'Sports, Exercise, and Recreation',
 't14': 'Religious and Spiritual Activities',
 't15': 'Volunteer Activities',
 't16': 'Telephone Calls',
 't18': 'Traveling',
 't50': 'Data Codes'}

In [28]:
average_minutes2(adults_with_no_children, "0101") / 60

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


8.750853706181001

In [29]:
pa.codes_dict['t0101']

'Sleeping'

In [30]:
average_minutes2(adults_with_no_children, "01") / 60

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


9.5541911543273592

In [31]:
# grouping
help(adults_with_no_children.groupby)

Help on method groupby in module pandas.core.generic:

groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False) method of pandas.core.frame.DataFrame instance
    Group series using mapper (dict or key function, apply given function
    to group, return result as series) or by a series of columns
    
    Parameters
    ----------
    by : mapping function / list of functions, dict, Series, or tuple /
        list of column names.
        Called on each element of the object index to determine the groups.
        If a dict or Series is passed, the Series or dict VALUES will be
        used to determine the groups
    axis : int, default 0
    level : int, level name, or sequence of such, default None
        If the axis is a MultiIndex (hierarchical), group by a particular
        level or levels
    as_index : boolean, default True
        For aggregated output, return object with group labels as the
        index. Only relevant for DataFrame inpu

## Joining files

In [32]:
respondents = open_atus_zip('atusresp_2013')
activities = open_atus_zip('atusact_2013')
# respondents = pd.read_csv("atusdata/atusresp_2013/atusresp_2013.dat")
# activities = pd.read_csv("atusdata/atusact_2013/atusact_2013.dat")

In [33]:
respondents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11385 entries, 0 to 11384
Columns: 175 entries, TUCASEID to TXTONHH
dtypes: float64(1), int64(172), object(2)
memory usage: 15.3+ MB


In [34]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215576 entries, 0 to 215575
Data columns (total 31 columns):
TUCASEID        215576 non-null int64
TUACTIVITY_N    215576 non-null int64
TEWHERE         215576 non-null int64
TRTCCTOT_LN     215576 non-null int64
TRTCC_LN        215576 non-null int64
TRTCOC_LN       215576 non-null int64
TRTEC_LN        215576 non-null int64
TRTHH_LN        215576 non-null int64
TRTNOHH_LN      215576 non-null int64
TRTOHH_LN       215576 non-null int64
TRTONHH_LN      215576 non-null int64
TRTO_LN         215576 non-null int64
TRWBELIG        215576 non-null int64
TUACTDUR        215576 non-null int64
TUACTDUR24      215576 non-null int64
TUCC5           215576 non-null int64
TUCC5B          215576 non-null int64
TUCC7           215576 non-null int64
TUCC8           215576 non-null int64
TUCUMDUR        215576 non-null int64
TUCUMDUR24      215576 non-null int64
TUDURSTOP       215576 non-null int64
TUEC24          215576 non-null int64
TUSTARTTIM     

In [35]:
respondents.head()

,TUCASEID,TULINENO,TUYEAR,TUMONTH,TEABSRSN,TEERN,TEERNH1O,TEERNH2,TEERNHRO,TEERNHRY,...,TXSPEMPNOT,TXSPUHRS,TXTCC,TXTCCTOT,TXTCOC,TXTHH,TXTNOHH,TXTO,TXTOHH,TXTONHH
0,20130101130004,1,2013,1,-1,-1,-1,-1,-1,-1,...,-1,-1,0,0,0,0,0,-1,-1,-1
1,20130101130112,1,2013,1,-1,-1,-1,-1,-1,2,...,0,0,0,0,0,0,-1,0,0,-1
2,20130101130123,1,2013,1,-1,-1,-1,-1,-1,-1,...,0,0,-1,0,0,-1,-1,-1,-1,-1
3,20130101130611,1,2013,1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,0,0,-1,-1,-1,-1,-1
4,20130101130616,1,2013,1,-1,-1,-1,-1,-1,2,...,-1,-1,-1,0,0,-1,-1,-1,-1,-1


In [36]:
activities.head()

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,TUDURSTOP,TUEC24,TUSTARTTIM,TUSTOPTIME,TUTIER1CODE,TUTIER2CODE,TUTIER3CODE,TRCODE,TRTIER2,TXWHERE
0,20130101130004,1,-1,0,0,0,-1,0,0,-1,...,2,-1,04:00:00,12:00:00,1,1,1,10101,101,0
1,20130101130004,2,1,5,5,0,-1,5,5,-1,...,1,-1,12:00:00,12:05:00,11,1,1,110101,1101,0
2,20130101130004,3,1,120,120,0,-1,120,120,-1,...,1,-1,12:05:00,14:05:00,12,3,3,120303,1203,0
3,20130101130004,4,1,0,0,0,-1,0,0,-1,...,2,-1,14:05:00,19:00:00,6,3,1,60301,603,0
4,20130101130004,5,1,30,30,0,-1,30,30,-1,...,1,-1,19:00:00,19:30:00,11,1,1,110101,1101,0


In [37]:
merged = pd.merge(respondents, activities, left_on="TUCASEID", right_on="TUCASEID")
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215576 entries, 0 to 215575
Columns: 205 entries, TUCASEID to TXWHERE
dtypes: float64(1), int64(200), object(4)
memory usage: 338.8+ MB


In [38]:
merged[["TUCASEID", "TUACTIVITY_N", "TXTCOC"]].head()

,TUCASEID,TUACTIVITY_N,TXTCOC
0,20130101130004,1,0
1,20130101130004,2,0
2,20130101130004,3,0
3,20130101130004,4,0
4,20130101130004,5,0


In [39]:
telfs = pd.Series({1: "Employed - at work",
                   2: "Employed - absent",
                   3: "Unemployed - laid off",
                   4: "Unemployed - looking",
                   5: "Not in labor force"})

In [40]:
telfs

1       Employed - at work
2        Employed - absent
3    Unemployed - laid off
4     Unemployed - looking
5       Not in labor force
dtype: object

In [41]:
summary.TELFS.map(telfs).head()

0      Not in labor force
1      Employed - at work
2    Unemployed - looking
3      Employed - at work
4      Employed - at work
Name: TELFS, dtype: object

##Correlation matrix

In [80]:
#import numpy as np
df = pd.DataFrame(np.random.random((5, 5)))
corr_series = df.corr().stack()
corr_series[(corr_series.map(math.fabs) >= 0.5) & (corr_series != 1.0)]


0  4    0.508207
1  3   -0.833739
   4    0.883459
3  1   -0.833739
   4   -0.922282
4  0    0.508207
   1    0.883459
   3   -0.922282
dtype: float64

#Verifying summary statistics from BLS website:
###http://www.bls.gov/tus/charts/home.htm

1. Americans >= 15 years old slept 8.7 hours, 
2. spent 5.3 hours doing leisure/sports activities
3. 3.5 hours working
4. 1.8 hours doing household activities.

In [42]:
s = summary3  # The dataframe with meaningful column names
[col for col in s.columns if not col[1].startswith('t')] # the summary demographic information about respondents

[('tucaseid', 'atus_case_id'),
 ('TUFINLWGT', 'atus_final_weight'),
 ('TRYHHCHILD', 'youngest_child_age'),
 ('TEAGE', 'age_edited'),
 ('TESEX', 'gender'),
 ('PEEDUCA', 'education_level'),
 ('PTDTRACE', 'race'),
 ('PEHSPNON', 'is_hispanic'),
 ('GTMETSTA', 'metropolitan_status'),
 ('TELFS', 'employment_status'),
 ('TEMJOT', 'has_multiple_jobs'),
 ('TRDPFTPT', 'work_status'),
 ('TESCHENR', 'is_student'),
 ('TESCHLVL', 'school_level'),
 ('TRSPPRES', 'partner_present'),
 ('TESPEMPNOT', 'partner_employed'),
 ('TRERNWA', 'weekly_earnings_main'),
 ('TRCHILDNUM', 'household_children'),
 ('TRSPFTPT', 'partner_work_status'),
 ('TEHRUSLT', 'weekly_hours_worked'),
 ('TUDIARYDAY', 'date'),
 ('TRHOLIDAY', 'is_holiday'),
 ('TRTEC', 'eldercare_minutes'),
 ('TRTHH', 'childcare_minutes'),
 ('t010101', 'Sleeping'),
 ('t010102', 'Sleeplessness'),
 ('t010201', 'Washing, dressing and grooming oneself'),
 ('t010299', 'Grooming, n.e.c.*'),
 ('t010301', 'Health-related self care'),
 ('t010399', 'Self care, n.e.

In [43]:
pa.categories  # The major categories of activities

{'t01': 'Personal Care',
 't02': 'Household Activities',
 't03': 'Caring For & Helping Household (HH) Members',
 't04': 'Caring For & Helping Nonhousehold (NonHH) Members',
 't05': 'Work & Work-Related Activities',
 't06': 'Education',
 't07': 'Consumer Purchases',
 't08': 'Professional & Personal Care Services',
 't09': 'Household Services',
 't10': 'Government Services & Civic Obligations',
 't11': 'Eating and Drinking',
 't12': 'Socializing, Relaxing, and Leisure',
 't13': 'Sports, Exercise, and Recreation',
 't14': 'Religious and Spiritual Activities',
 't15': 'Volunteer Activities',
 't16': 'Telephone Calls',
 't18': 'Traveling',
 't50': 'Data Codes'}

In [44]:
pa.sub_categories['t0101']

'Sleeping'

In [45]:
avg_american_crit = summary.TEAGE >= 15
american = s[avg_american_crit]
a = american
b = a
b.columns = a.columns.droplevel(1)  # To remove MultiIndex
a.head()

,tucaseid,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20130101130004,11899905.662034,12,22,2,40,8,2,1,5,...,0,0,0,0,0,0,0,0,0,0
1,20130101130112,4447638.009513,1,39,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
2,20130101130123,10377056.507734,-1,47,2,40,1,2,1,4,...,25,0,0,0,0,0,0,0,0,0
3,20130101130611,7731257.992805,-1,50,2,40,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20130101130616,4725269.227067,-1,45,2,40,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0


In [46]:
def search_codes(text):
    results = []
    for k,v in pa.codes_dict.items():
        #print(v)
        if re.search(text, v.lower()):
            results.append((k,v))
    return results
def pick_activities(text, desc=False):
    """
    Returns activity codes with text in description
    If desc=True, returns the description instead
    """
    if desc:
        item = 1
    else:
        item = 0
    return [i[item] for i in search_codes(text) if len(i[0])==7]
    
search_codes('sleep')
sleep_activities = pick_activities('sleep')
sleep_activities = sleep_activities[1:]  ##t010199 not in summary for some reason
sleep_activities


['t010102', 't010101']

In [47]:
def sum_cols(col_list):
    pass

In [48]:
#c = b.groupby('tucaseid')[sleep_activities]
c = b.set_index('tucaseid')
d = c[sleep_activities].sum(axis=1) * c.TUFINLWGT
d.sum() / c.TUFINLWGT.sum() / 60
# [sleep_activities.extend]#.groupby('tucaseid')

8.7381765406301852

^^ Test 1/4 passed ^^

In [49]:
search_codes('leisure')  # t12----
search_codes('sports')   # t13----
#pick_activities('leisure')

[('t181399', 'Travel related to sports, exercise, & recreation, n.e.c.*'),
 ('t130399',
  'Waiting associated with sports, exercise, & recreation, n.e.c.*'),
 ('t030103', 'Playing with hh children, not sports'),
 ('t130132', 'Participating in water sports'),
 ('t130199', 'Playing sports n.e.c.*'),
 ('t1304', 'Security Procedures Rel. to Sports, Exercise, & Recreation'),
 ('t130218', 'Watching racquet sports'),
 ('t130499', 'Security related to sports, exercise, & recreation, n.e.c.*'),
 ('t1204', 'Arts and Entertainment (other than sports)'),
 ('t050203', 'Sports and exercise as part of job'),
 ('t130401', 'Security related to playing sports or exercising'),
 ('t139999', 'Sports, exercise, & recreation, n.e.c.*'),
 ('t0602', 'Extracurricular School Activities (Except Sports)'),
 ('t040105', 'Playing sports with nonhh children'),
 ('t130210', 'Watching equestrian sports'),
 ('t130120', 'Playing racquet sports'),
 ('t130229', 'Watching water sports'),
 ('t180602', 'Travel related to extr

In [50]:
snl = activity_columns(b, "12")
snl.extend(activity_columns(b, "13"))
#snl.extend(['t030105'])
e = b.set_index('tucaseid')
f = e[snl].rename(columns=pa.codes_dict)
g = f.sum(axis=1) * e.TUFINLWGT
g.sum() / c.TUFINLWGT.sum() / 60
# Matches: average_minutes2(b, "12") / 60 + average_minutes2(b, "13") / 60

5.0343812445470784

^^ 2/4 (close, but got 5.04 instead of 5.3...) ^^

In [51]:
search_codes('work')  # t05
average_minutes2(b, "05") / 60

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


3.2018469853082667

^^ close to 3.5 hours expected ^^

In [52]:
pa.categories['t02']
average_minutes2(b, "02") / 60

/usr/local/lib/python3.4/site-packages/IPython/kernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1.7825438820657689

^^ 4/4 matches 1.8 hours ^^

##Now for some basic analyses using the summary data
###Let's define some useful selection criteria

In [53]:
list(zip(meanings_unsorted, summary.columns[:24]))

[('atus_case_id', 'tucaseid'),
 ('atus_final_weight', 'TUFINLWGT'),
 ('youngest_child_age', 'TRYHHCHILD'),
 ('age_edited', 'TEAGE'),
 ('gender', 'TESEX'),
 ('education_level', 'PEEDUCA'),
 ('race', 'PTDTRACE'),
 ('is_hispanic', 'PEHSPNON'),
 ('metropolitan_status', 'GTMETSTA'),
 ('employment_status', 'TELFS'),
 ('has_multiple_jobs', 'TEMJOT'),
 ('work_status', 'TRDPFTPT'),
 ('is_student', 'TESCHENR'),
 ('school_level', 'TESCHLVL'),
 ('partner_present', 'TRSPPRES'),
 ('partner_employed', 'TESPEMPNOT'),
 ('weekly_earnings_main', 'TRERNWA'),
 ('household_children', 'TRCHILDNUM'),
 ('partner_work_status', 'TRSPFTPT'),
 ('weekly_hours_worked', 'TEHRUSLT'),
 ('date', 'TUDIARYDAY'),
 ('is_holiday', 'TRHOLIDAY'),
 ('eldercare_minutes', 'TRTEC'),
 ('childcare_minutes', 'TRTHH')]

    summary.groupby('TRYHHCHILD').size()[1:].plot(kind='bar',title='Youngest Child Age')
### Youngest child ranges from 0 to 17, with > 200 observations at each age

In [54]:
adults = summary[summary.TEAGE > 17]
newborn_parents = adults[adults.TRYHHCHILD == 0].set_index('tucaseid')
non_parents = adults[adults.TRYHHCHILD == -1].set_index('tucaseid')
# dir(summary.groupby('TRYHHCHILD'))
# summary.groupby('TRYHHCHILD')
cols = [col for col in newborn_parents]
# for col in cols[2:]:
#     newborn_parents[col] = weighted_col(newborn_parents, col)
# newborn_parents['t010101'] = weighted_col(newborn_parents, 't010101')
# newborn_parents['t010101']
# newborn_parents

In [55]:
nonp_sleeping = average_minutes(non_parents, '010101')
newp_sleeping = average_minutes(newborn_parents, '010101')
nonp_sleeping/newp_sleeping
# This does not make sense!

1.0019294840010831

###Quick Analysis -- What is the maximum time anyone spends on each category?

In [56]:
s = summary.set_index('tucaseid')

def one_track_minds(s):
    """Returns users with maximum for each activity code, requires s summary dataframe"""
    s = summary.set_index('tucaseid')
    # s.t010101.max()/60
    # s.t010101.idxmax()
    extreme_ppl = []
    for activity in s.columns[23:]:
        person = s[activity].idxmax()
        minutes = s.loc[person, activity]
        extreme_ppl.append((person, minutes/60, activity, pa.codes_dict[activity]))
    xp = pd.DataFrame(extreme_ppl, columns=['tucaseid', 'hours', 'activity', 'description']).set_index('tucaseid')
    #xp.sort('minutes')  # Some of the least popular activities include "Public health activities, and several n.e.c* labels
    xp = xp.sort('hours', ascending=False)
    return xp
xp = one_track_minds(s)
# [user_report(s,i[0]) for i in xp.iloc[0:50].iterrows()]

In [57]:
def user_report(df, user=20131110131406):
    """Takes a dataframe of activity times and returns hours spend per day for each non-zero value"""
    stats = df.loc[user][:23]
    time = df.loc[user][23:]
    time_use = time[time != 0].rename(pa.codes_dict)
    hours = pd.DataFrame((time_use / 60))
    bio = pd.DataFrame(stats).rename(summary_columns)
#     hours.plot(kind='pie', subplots=True)
    return hours, bio


In [58]:
for user in [user for user in xp.index][:20]:
    hours, bio = user_report(s, user)
plt.figure(figsize=(20,10))# hours.plot(kind='pie', subplots=True)
# plt.plot?

In [59]:
xp = one_track_minds(s)

In [77]:
from IPython.display import display
pd.options.display.max_columns = 30 #20 by default

reports = [user_report(s,i[0]) for i in xp.iloc[0:10].iterrows()]
xps = pd.DataFrame()
cc = 1
for i, val in enumerate(reports):
    bb = reports[i][1].stack().unstack(0)
    bb.index.names=['tucaseid']
    cc = pd.merge(xp, bb, left_index=True, right_index=True)
    xps = xps.append(cc)
xps
# cc.stack()
# bb = bio.stack().unstack(0)
# bb.index.names=['tucaseid']
# pd.merge(xp, bb, left_index=True, right_index=True)

,hours,activity,description,atus_final_weight,youngest_child_age,age_edited,gender,education_level,race,is_hispanic,metropolitan_status,employment_status,has_multiple_jobs,work_status,is_student,school_level,partner_present,partner_employed,weekly_earnings_main,household_children,partner_work_status,weekly_hours_worked,date,is_holiday,eldercare_minutes,childcare_minutes
tucaseid,,,,,,,,,,,,,,,,,,,,,,,,,,
20131110131406,23.550000,t010101,Sleeping,1899058.155655,-1,52,2,39,2,2,1,1,2,1,-1,-1,3,-1,48000,0,-1,40,7,0,27,0
20130807132371,22.500000,t050101,"Work, main job",3107214.726310,9,47,1,43,1,1,1,1,2,1,2,-1,1,1,192307,2,1,45,1,0,-1,0
20130807132365,22.000000,t500101,Insufficient detail in verbatim,3345022.104425,2,32,1,40,1,2,1,1,2,1,2,-1,1,2,288461,2,-1,99,1,0,-1,0
20131009131778,21.816667,t120303,Television and movies (not religious),3495526.270281,-1,63,1,38,2,2,1,4,-1,-1,-1,-1,3,-1,-1,0,-1,-1,4,0,-1,0
20130605131901,21.050000,t120312,Reading for personal interest,2218752.415573,-1,85,1,40,1,2,1,5,-1,-1,-1,-1,3,-1,-1,0,-1,-1,7,0,-1,0
20130201131342,20.500000,t030401,Physical care for hh adults,4884344.681430,-1,58,2,43,1,2,1,2,2,2,-1,-1,3,-1,76923,0,-1,12,1,0,-1,0
20130212121071,19.716667,t120301,"Relaxing, thinking",6002226.550923,-1,80,1,38,1,2,1,5,-1,-1,-1,-1,3,-1,-1,0,-1,-1,3,0,-1,0
20130302130037,19.583333,t020103,"Sewing, repairing, & maintaining textiles",1940694.543983,-1,65,2,36,1,2,2,1,2,2,-1,-1,3,-1,10000,0,-1,10,1,1,-1,0
20130908131014,19.000000,t030101,Physical care for hh children,7872288.902837,0,30,2,43,1,2,1,1,2,2,2,-1,1,1,30000,1,1,15,5,0,-1,0


##Further Exploration

###Possible Questions:
- What are the most extreme uses of the hours in a day?
- Newborn parent sleep - dads vs moms
- Amount of lawn work vs age of youngest child
- Amount of alone time vs age of youngest child TRTALONE
- What are people doing at this moment in time?
- What did people who responded "none of your business" 500105 do otherwise?
- Show a random correlation
- What do sleepless people do?
- OTPYG, by the numbers
- Rodeo vs ???
- Organizing and Preparing vs ???
- What do people do on holiday vs day off vs day of work? Weekend vs weekday?

###Possible groupings:
- Adults with and without children
- Age of youngest child (0-5, 6-11, 12-17)
- Age <25, <40, <65, >65
- Male/Female
- 

###Other variables/files of interest:
- Actual activities file

###Discussion Points:
- Generating data dictionary from pdf
- MadLib/User report

###Other:
- Critical Functions:
    - dictionaries in pa
    - search_codes
    - pick_codes
    - user_report
- Test suite

- Compare different populations (people with children and people without, people of differing age groups, men and women, or other groupings)
Answer macro-level questions and micro-level questions (for example, the amount of leisure for the macro-level, the types of things people do for leisure for the micro-level)
Your final analysis should be in the form of an IPython Notebook with both narrative analysis and supporting charts. Your supporting code should be in normal Python files.


In [61]:
search_codes

<function __main__.search_codes>

#Table of Contents:
* [Initial Exploration (Top)](#Initial-Exploration)
* [Table of Contents](#Table-of-Contents:)
* [Relevant Links](#Relevant-links:)
* [Joining Files](#Joining-files)
* [Correlation Matrix](#Correlation-matrix)
* [Verifying BLS Statistics](#Verifying-summary-statistics-from-BLS-website:)
* [Further Analyses](#Now-for-some-basic-analyses-using-the-summary-data)
    * [One-track-minds](#Quick-Analysis----What-is-the-maximum-time-anyone-spends-on-each-category?)
* [Further Exploration](#Further-Exploration)
* [End](#End)

##End